
<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/11_Jacobian_and_Hessian.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [ ]:
# --------------------------------------------------------------
# ☁️ COLAB SETUP (Automatyczna instalacja środowiska)
# --------------------------------------------------------------
import sys
import os

# Sprawdzamy, czy jesteśmy w Google Colab
if 'google.colab' in sys.modules:
    print('☁️ Wykryto środowisko Google Colab. Konfiguruję...')

    # 1. Pobieramy plik requirements.txt bezpośrednio z repozytorium
    !wget -q https://raw.githubusercontent.com/takzen/ai-engineering-handbook/main/requirements.txt -O requirements.txt

    # 2. Instalujemy biblioteki
    print('⏳ Instaluję zależności (to może chwilę potrwać)...')
    !pip install -q -r requirements.txt

    print('✅ Gotowe! Środowisko jest zgodne z repozytorium.')
else:
    print('💻 Wykryto środowisko lokalne. Zakładam, że masz już uv/venv.')


# 🥋 Lekcja 11: Jakobian i Hessian (Pochodne Wyższego Rzędu)

Standardowe `backward()` działa tylko wtedy, gdy wynik jest **skalarem** (jedną liczbą, np. Loss).
Gdy wynik jest wektorem, PyTorch oblicza tzw. **vJP (vector-Jacobian Product)**, czyli od razu mnoży macierz pochodnych przez wektor gradientu z góry.

Ale czasem potrzebujemy **pełnej macierzy**:
1.  **Jakobian (J):** Macierz pierwszych pochodnych dla funkcji wektorowych ($R^n \to R^m$).
2.  **Hessian (H):** Macierz drugich pochodnych ($R^n \to R$). Mówi nam o **krzywiźnie** powierzchni błędu.

**Zastosowanie:**
*   **MAML:** Optymalizacja parametrów optymalizatora.
*   **Influence Functions:** Sprawdzanie, który przykład treningowy najbardziej wpłynął na decyzję modelu.
*   **Newton's Method:** Szybsza optymalizacja (zamiast zgadywać krok, skaczemy prosto do minimum paraboli).

In [1]:
import torch
from torch.autograd.functional import jacobian, hessian

# Prosta funkcja wektorowa: wejście (x, y) -> wyjście (x^2, y^3)
def func_vector(inputs):
    x, y = inputs[0], inputs[1]
    return torch.stack([x**2, y**3])

# Prosta funkcja skalarna: wejście (x, y) -> wyjście x^2 + y^2 (Miseczka)
def func_scalar(inputs):
    x, y = inputs[0], inputs[1]
    return x**2 + y**2

inputs = torch.tensor([2.0, 2.0])

print("Funkcje zdefiniowane.")
print(f"Dla wejścia {inputs}:")
print(f"Wynik wektorowy: {func_vector(inputs)}") # [4, 8]
print(f"Wynik skalarny:  {func_scalar(inputs)}") # 8

Funkcje zdefiniowane.
Dla wejścia tensor([2., 2.]):
Wynik wektorowy: tensor([4., 8.])
Wynik skalarny:  8.0


## 1. Macierz Jakobianu

Funkcja: $f(x, y) = [x^2, y^3]$

Jakobian to macierz pochodnych cząstkowych wszystkich wyjść po wszystkich wejściach:
$$
J = \begin{bmatrix}
\frac{\partial f_1}{\partial x} & \frac{\partial f_1}{\partial y} \\
\frac{\partial f_2}{\partial x} & \frac{\partial f_2}{\partial y}
\end{bmatrix} = \begin{bmatrix}
2x & 0 \\
0 & 3y^2
\end{bmatrix}
$$

Dla $x=2, y=2$:
$$
J = \begin{bmatrix}
4 & 0 \\
0 & 12
\end{bmatrix}
$$

In [2]:
# Obliczamy Jakobian automatycznie
J = jacobian(func_vector, inputs)

print("--- MACIERZ JAKOBIANU ---")
print(J)

# Weryfikacja
expected = torch.tensor([[4., 0.], 
                         [0., 12.]])

if torch.allclose(J, expected):
    print("✅ Matematyka się zgadza!")

--- MACIERZ JAKOBIANU ---
tensor([[ 4.,  0.],
        [ 0., 12.]])
✅ Matematyka się zgadza!


## 2. Macierz Hessianu (Druga Pochodna)

Hessian mówi nam, jak bardzo "zakrzywiona" jest funkcja. Jest kluczowy, by wiedzieć, czy jesteśmy w dołku (minimum), na górce (maksimum) czy na siodle.

Funkcja: $f(x, y) = x^2 + y^2$
Pierwsza pochodna (Gradient): $[2x, 2y]$
Druga pochodna (Hessian):
$$
H = \begin{bmatrix}
\frac{\partial^2 f}{\partial x^2} & \frac{\partial^2 f}{\partial x \partial y} \\
\frac{\partial^2 f}{\partial y \partial x} & \frac{\partial^2 f}{\partial y^2}
\end{bmatrix} = \begin{bmatrix}
2 & 0 \\
0 & 2
\end{bmatrix}
$$

Zauważ, że Hessian jest stały (nie zależy od x, y), bo funkcja to idealna parabola.

In [3]:
# Obliczamy Hessian
H = hessian(func_scalar, inputs)

print("--- MACIERZ HESSIANU ---")
print(H)

# Analiza krzywizny
# Wartości własne Hessianu mówią o kształcie
eigenvalues = torch.linalg.eigvalsh(H)
print(f"\nWartości własne: {eigenvalues}")

if (eigenvalues > 0).all():
    print("👉 Dodatnio określony: Jesteśmy w 'miseczce' (lokalne minimum).")
elif (eigenvalues < 0).all():
    print("👉 Ujemnie określony: Jesteśmy na 'górce' (lokalne maksimum).")
else:
    print("👉 Mieszane: Punkt siodłowy.")

--- MACIERZ HESSIANU ---
tensor([[2., 0.],
        [0., 2.]])

Wartości własne: tensor([2., 2.])
👉 Dodatnio określony: Jesteśmy w 'miseczce' (lokalne minimum).


## Inżynieryjne Ostrzeżenie (Pamięć!)

Dlaczego nie używamy tego w każdym treningu?
Bo Hessian ma rozmiar $N \times N$, gdzie $N$ to liczba parametrów modelu.

Jeśli Twój model ma 1 milion parametrów (malutki):
*   Gradient: 1 mln liczb (4 MB).
*   Hessian: $10^{12}$ liczb (4 TB!). **To się nie zmieści w żadnym komputerze.**

Dlatego w Deep Learningu używa się triku **Hessian-Vector Product (HvP)**.
Możemy policzyć iloczyn $H \cdot v$ bez obliczania całej macierzy $H$!
PyTorch robi to automatycznie, gdy policzysz gradient z gradientu.

In [4]:
# Przykład HvP (bez tworzenia wielkiej macierzy)

x = torch.tensor([2.0, 2.0], requires_grad=True)
y = x[0]**2 + x[1]**2  # x^2 + y^2

# 1. Pierwszy Gradient (create_graph=True pozwala liczyć dalej!)
grads = torch.autograd.grad(y, x, create_graph=True)[0]
print(f"Gradient (pierwsza pochodna): {grads}") # [4, 4]

# 2. Drugi Gradient (rzutowany na wektor v)
# Załóżmy v = [1, 1]
v = torch.tensor([1.0, 1.0])
# Liczymy pochodną z (Gradient * v)
# To daje nam H * v
hvp = torch.autograd.grad(grads, x, grad_outputs=v)[0]

print(f"Hessian-Vector Product (H*v): {hvp}")
# H to [[2,0],[0,2]], v to [1,1]. Wynik powinien być [2, 2].

Gradient (pierwsza pochodna): tensor([4., 4.], grad_fn=<AddBackward0>)
Hessian-Vector Product (H*v): tensor([2., 2.])


## 🥋 Black Belt Summary

1.  **`jacobian` i `hessian`**: Świetne do analizy matematycznej małych funkcji lub debugowania. Bezużyteczne dla pełnych sieci neuronowych (przez pamięć).
2.  **`create_graph=True`**: Klucz do liczenia pochodnych wyższego rzędu. Mówi PyTorchowi: "Graf, który właśnie zbudowałeś do policzenia gradientu? Nie wyrzucaj go! Chcę go różniczkować jeszcze raz".
3.  **HvP (Hessian-Vector Product):** To jedyny sposób na używanie informacji o krzywiźnie w dużych sieciach (np. w metodach optymalizacji drugiego rzędu).